In [1]:
from IPython.display import HTML

import math
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_notebook, show

output_notebook()

Loading BokehJS ...

In [2]:
import sys
sys.path.insert(1,'../covid-website')

#import SISV as s
import ContactRate as cr


In [3]:
nE   = 6  #maximum number of "Exposed" stages (incubation period)
nI   = 6  #maximum number of "Infectious" stages
nC   = 6  #maximum number of "Critical" stages (fatalities)
nT   = 6  #maximum number ot "Testing" stages  

cS   = 0  #Susceptible people
cE   = 1
cE1  = 2
cE2  = 3
cE3  = 4
cE4  = 5
cE5  = 6
cI   = 7  #total infectious 
cI0  = 8  #Infectious stage 1
cI1  = 9
cI2  = 10 
cI3  = 11
cI4  = 12
cI5  = 13
cR   = 14  #Recovered after infectious (cumulative)
cV   = 15  #Vaccinated
cT   = 16  #Testing: a fraction of infectious people will test positive; results will be available with a delay
cT1  = 17
cT2  = 18
cT3  = 19
cT4  = 20
cT5  = 21
cC   = 22  #Critical: seriously ill after end of infectious period, will die; assume isolated so not contagious
cC1  = 23
cC2  = 24
cC3  = 25
cC4  = 26
cC5  = 27
cF   = 28  #cumulative fatalities
cP   = 29  #cumulative tested positive
cNum = 30


cCols = [
"S",
"E",
"E1",
"E2",
"E3",
"E4",
"E5",
"I",
"I0",
"I1",
"I2",
"I3",
"I4",
"I5",
"R",
"V",
"T",
"T1",
"T2",
"T3",
"T4",
"T5",
"C",
"C1",
"C2",
"C3",
"C4",
"C5",
"F",
"P",
]



def SISV(x, params):

    population          = params['population']

    exp_stages          = params['exp_stages']              #number of incubation stages [0,nE]
    inf_stages          = params['inf_stages']              #number of infectious stages [0,nI]
    crit_stages         = params['crit_stages']             #number of critical stages [0,nC]
    test_stages         = params['test_stages']             #number of testing stages [0,nT]
    
    death_rate          = params['death_rate']              #Infection Fatality Rate
    gamma_exp           = params['gamma_exp']               #1/test result delay
    gamma               = params['gamma']                   #1/length of infectious period
    gamma_pos           = params['gamma_pos']               #1/test result delay
    gamma_crit          = params['gamma_crit']              #1/(critical care + death reporting time)
    detection_rate      = params['detection_rate']          #percentage of infectious people that test positive
    immun               = params['immun']                   #1/length of natural immunity after recovery
    vacc_start          = params['vacc_start']
    vacc_rate           = params['vacc_rate']               #fraction of susceptible population vaccinated per day
    vacc_immun          = params['vacc_immun']              #1/length of immunity by vaccination
    
    i0                  = params['i0']
    #f0                  = params['f0']
    beta0               = params['beta0']
    #c0                  = params['c0']
    
    y = np.zeros((x.size, cNum))
    
    y[0, cE]  = i0 * math.sqrt(beta0/gamma)
    y[0, cI]  = i0
    y[0, cI0] = i0
    y[0, cC]  = i0 * death_rate * gamma / (beta0+gamma_crit-gamma)
    
    #y[0, cR] = f0 / death_rate
    #y[0, cF] = 0 if "f0" not in params else params['f0']
    #y[0, cP] = 0 if "p0" not in params else params['p0']
    #y[0, cS] = population - y[0,cI] if "s0" not in params else params['s0']
    #y[0, cR] = population - (y[0, cS]+y[0,cI]+y[0, cF])
    y[0, cS] = population - y[0,cE] - y[0,cI] - y[0,cC] - y[0,cF] - y[0,cR]
    
    #the contact rate beta depends on time
    interv = cr.contact_rate(x, params)   
    

    for i in range(1, x.size):

        #calculate the flows from one compartment to another at each time step
        
        flows = [] #list of tuples; each tuple is a flow from one bucket to another (source, dest, amount)

        #infection
        beta = interv[i-1]
        #newlyinfectious = beta * y[i-1, cS] * y[i-1, cI] / population 
        #flows.append((cS, cI0, newlyinfectious))  #move to first infectious stage cI0
        
        newlyexposed = beta * y[i-1, cS] * y[i-1, cI] / population 
        
        if exp_stages==0:
            newlyinfectious = newlyexposed
            flows.append((cS, cI0, newlyinfectious))  #move to first stage of infectious period, no incubation
        else:
            flows.append((cS, cE, newlyexposed))  #move to first stage of incubation period cE
        
            #several stages of incubation change the proba distribution from exponential to gamma
            #first stage from cE ; last stage to cE=cE+nE
            for j in range(exp_stages):  #exp_stages must be in [1,nE]
                exp_i = exp_stages * gamma_exp * y[i-1, cE+j]
                if j==exp_stages-1: #last stage moves to cI0
                    newlyinfectious = exp_i #will be needed for the Testing pathway
                    flows.append((cE+j, cI0, newlyinfectious)) 
                else: #move to next incubation stage
                    flows.append((cE+j, cE+j+1, exp_i)) 
                
        #several infectious stages to change the proba distribution from exponential to gamma shape
        #first stage from cI0 ; last stage to cI0+inf_stages-1
        for j in range(inf_stages-1):  #inf_stages must be in [0,nI]
            inf_i = inf_stages * gamma * y[i-1, cI0+j]
            flows.append((cI0+j, cI0+j+1, inf_i)) 
        
        leave_infectious = inf_stages * gamma * y[i-1, cI0+inf_stages-1]
        
        #a portion of infectious people go into critical care for a while (and will eventually die)
        newlycritical = death_rate * leave_infectious
        flows.append((cI0+inf_stages-1, cC, newlycritical))

        #the rest of infectious people recover
        newlyrecovered = (1-death_rate) * leave_infectious
        flows.append((cI0+inf_stages-1, cR, newlyrecovered))

        #several stages of critical care to change the proba distribution from exponential to gamma
        #first stage from cC ; last stage to cF=cI+nc
        for j in range(crit_stages):  #crit_stages must be in [0,nC]
            crit_i = crit_stages * gamma_crit * y[i-1, cC+j]
            if j==crit_stages-1: #last stage moves to cF
                flows.append((cC+j, cF, crit_i)) 
            else: #move to next critical stage
                flows.append((cC+j, cC+j+1, crit_i)) 
        
        #people in critical care die and their death is reported
        #newlydead = gamma_crit * y[i-1, cC]
        #flows.append((cC, cF, newlydead))
        
        #loss of natural immunity after recovery from exposure
        newlysusceptible_r = immun * y[i-1, cR]
        flows.append((cR, cS, newlysusceptible_r))

        #vaccination
        newlyvaccinated = vacc_rate * y[i-1, cS] if i >= vacc_start else 0
        flows.append((cS, cV, newlyvaccinated))
        
        #loss of immunity after vaccination
        newlysusceptible_v = vacc_immun * y[i-1, cV]
        flows.append((cV, cS, newlysusceptible_v))
        
        #testing of infectious people
        newlytested = detection_rate * newlyinfectious
        flows.append((-1, cT, newlytested))  #-1 to avoid removing this flow from its source

        #several stages of testing to change the proba distribution from exponential to gamma
        #first stage from cT ; last stage to cT+test_stages-1
        for j in range(test_stages):  #test_stages must be in [0,nT]
            test_i = test_stages * gamma_pos * y[i-1, cT+j]
            if j==test_stages-1: #last stage moves to cP
                flows.append((cT+j, cP, test_i)) 
            else: #move to next critical stage
                flows.append((cT+j, cT+j+1, test_i)) 
        
        #publication of test results after a delay
        #newpositives = gamma_pos * y[i-1, cT]
        #flows.append((cT, cP, newpositives))
        
        #apply the flows to the source and destination compartments
        y[i] = y[i-1].copy()
        for (source, dest, flow) in flows:
            y[i,source] -= flow if source>-1 else 0  
            y[i, dest] += flow if dest>-1 else 0

        #tabulate total of all infectious stages and store in cI; will be used to calculate newlyinfectious
        y[i, cI] = 0 #previous row was copied when tabulating flows (see above)
        for j in range(inf_stages):  #inf_stages must be in [0,nI]
            y[i, cI] += y[i, cI0+j]

        
    return y


In [4]:
#for debugging, select the columns to print
#cols = ['S', 'E', 'E1','I', 'I1', 'R', 'C', 'F']

def jacobian_cols(params):
    
    exp_stages          = params['exp_stages']              #number of incubation stages [0,nE]
    inf_stages          = params['inf_stages']              #number of infectious stages [0,nI]
    crit_stages         = params['crit_stages']             #number of critical stages [0,nC]
    
    cols = ['S']
    if exp_stages>0:
        cols.append('E')
        for i in range(1, exp_stages):
            cols.append('E{}'.format(i))

    for i in range(0, inf_stages):
        cols.append('I{}'.format(i))

    cols.append('C')
    for i in range(1, crit_stages):
        cols.append('C{}'.format(i))
        
    cols.append('F')
    cols.append('R')

    return cols
#---------------------------------------------------------
def jacobian(params, S, beta):
    
    population          = params['population']

    exp_stages          = params['exp_stages']              #number of incubation stages [0,nE]
    inf_stages          = params['inf_stages']              #number of infectious stages [0,nI]
    crit_stages         = params['crit_stages']             #number of critical stages [0,nC]
    test_stages         = params['test_stages']             #number of testing stages [0,nT]
    
    death_rate          = params['death_rate']              #Infection Fatality Rate
    gamma_exp           = params['gamma_exp']               #1/test result delay
    gamma               = params['gamma']                   #1/length of infectious period
    gamma_pos           = params['gamma_pos']               #1/test result delay
    gamma_crit          = params['gamma_crit']              #1/(critical care + death reporting time)
    detection_rate      = params['detection_rate']          #percentage of infectious people that test positive
    immun               = params['immun']                   #1/length of natural immunity after recovery
    vacc_start          = params['vacc_start']
    vacc_rate           = params['vacc_rate']               #fraction of susceptible population vaccinated per day
    vacc_immun          = params['vacc_immun']              #1/length of immunity by vaccination

    J = np.zeros((cNum, cNum))

    #-------------
    if exp_stages>0:
        
        newlyexposed = beta * S / population
        for j in range(0, inf_stages):
            J[cE, cI0+j] = newlyexposed
            J[cS, cI0+j] = - newlyexposed

        for j in range(1,exp_stages):
            J[cE+j, cE+j-1] = exp_stages * gamma_exp

        for j in range(0,exp_stages):
            J[cE+j, cE+j] -= exp_stages * gamma_exp

        newlyinfectious = exp_stages * gamma_exp
        J[cI0, cE+exp_stages-1] = newlyinfectious
        J[cT, cE+exp_stages-1] = detection_rate * newlyinfectious

    else:
        newlyinfectious = beta * S / population
        for j in range(0, inf_stages):
            J[cI0, cI0+j] = newlyinfectious
            J[cS, cI0+j] = - newlyinfectious
            J[cT, cI0+j] = detection_rate * newlyinfectious

    #---------------
    for j in range(1,inf_stages):
        J[cI0+j, cI0+j-1] = inf_stages * gamma

    for j in range(0, inf_stages):
        J[cI0+j, cI0+j] -= inf_stages * gamma

    J[cR, cI0+inf_stages-1] = (1-death_rate) * inf_stages * gamma

    #---------------
    J[cC, cI0+inf_stages-1] = death_rate * inf_stages * gamma

    for j in range(1,crit_stages):
        J[cC+j, cC+j-1] = crit_stages * gamma_crit

    for j in range(0, crit_stages):
        J[cC+j, cC+j] -= crit_stages * gamma_crit

    J[cF, cC+crit_stages-1] = crit_stages * gamma_crit

    #------------
    for j in range(1, test_stages):
        J[cT+j, cT+j-1] = test_stages * gamma_pos

    for j in range(0, test_stages):
        J[cT+j, cT+j] -= test_stages * gamma_pos

    J[cP, cT+test_stages-1] = test_stages * gamma_pos

    #-------------
    J[cS, cR] += immun
    J[cR, cR] -= immun

    #-------------
    #if ti>=vacc_start:
    #    J[cV, cS] = vacc_rate
    #    J[cS, cS] -= vacc_rate

    #    J[cS, cV] = vacc_immun
    #    J[cV, cV] = - vacc_immun

    J[cI, cI] = 0
    for j in range(0, inf_stages):
        J[cI, cI0+j] = J[cI0+j, cI0+j]
        J[cI, cI] += J[cI0+j, cI]
        
    #pJ = pd.DataFrame(J, columns=cCols, index=cCols)
    #print(pJ[cols].loc[cols])
    
    return J

  

def SISV_J(x, params):

    population          = params['population']

    exp_stages          = params['exp_stages']              #number of incubation stages [0,nE]
    inf_stages          = params['inf_stages']              #number of infectious stages [0,nI]
    crit_stages         = params['crit_stages']             #number of critical stages [0,nC]
    test_stages         = params['test_stages']             #number of testing stages [0,nT]
    
    death_rate          = params['death_rate']              #Infection Fatality Rate
    gamma_exp           = params['gamma_exp']               #1/test result delay
    gamma               = params['gamma']                   #1/length of infectious period
    gamma_pos           = params['gamma_pos']               #1/test result delay
    gamma_crit          = params['gamma_crit']              #1/(critical care + death reporting time)
    detection_rate      = params['detection_rate']          #percentage of infectious people that test positive
    immun               = params['immun']                   #1/length of natural immunity after recovery
    vacc_start          = params['vacc_start']
    vacc_rate           = params['vacc_rate']               #fraction of susceptible population vaccinated per day
    vacc_immun          = params['vacc_immun']              #1/length of immunity by vaccination
    
    i0                  = params['i0']
    #f0                  = params['f0']
    beta0               = params['beta0']
    #c0                  = params['c0']
    
    y = np.zeros((x.size, cNum))
    
    if exp_stages>0:
        y[0, cE]  = i0 * math.sqrt(beta0/gamma) / exp_stages
        for i in range(1, exp_stages):
            y[0,cE+i] = y[0,cE]
            
    y[0, cI]  = i0 
    for i in range(0, inf_stages):
        y[0,cI0+i] = i0/inf_stages
    
    y[0, cC]  = i0 * death_rate * gamma / (beta0+gamma_crit-gamma) / crit_stages
    for i in range(1, crit_stages):
        y[0,cC+i] = y[0, cC]
        
    y[0, cS] = population - y[0,cE]*exp_stages - y[0,cI] - y[0,cC]*crit_stages - y[0,cF] - y[0,cR]
    
    #the contact rate beta depends on time
    interv = cr.contact_rate(x, params)   
    
    #---------------
    #---------------
    J = np.zeros((cNum, cNum))

    #-------------
    if exp_stages>0:

        for j in range(1,exp_stages):
            J[cE+j, cE+j-1] = exp_stages * gamma_exp

        for j in range(0,exp_stages):
            J[cE+j, cE+j] -= exp_stages * gamma_exp

        newlyinfectious = exp_stages * gamma_exp
        J[cI0, cE+exp_stages-1] = newlyinfectious
        J[cT, cE+exp_stages-1] = detection_rate * newlyinfectious

    
    #---------------
    for j in range(1,inf_stages):
        J[cI0+j, cI0+j-1] = inf_stages * gamma

    for j in range(0, inf_stages):
        J[cI0+j, cI0+j] -= inf_stages * gamma

    J[cR, cI0+inf_stages-1] = (1-death_rate) * inf_stages * gamma
    
    #---------------
    J[cC, cI0+inf_stages-1] = death_rate * inf_stages * gamma

    for j in range(1,crit_stages):
        J[cC+j, cC+j-1] = crit_stages * gamma_crit

    for j in range(0, crit_stages):
        J[cC+j, cC+j] -= crit_stages * gamma_crit

    J[cF, cC+crit_stages-1] = crit_stages * gamma_crit

    #------------
    for j in range(1, test_stages):
        J[cT+j, cT+j-1] = test_stages * gamma_pos

    for j in range(0, test_stages):
        J[cT+j, cT+j] -= test_stages * gamma_pos

    J[cP, cT+test_stages-1] = test_stages * gamma_pos

    #-------------
    J[cS, cR] += immun
    J[cR, cR] -= immun
        
    
    #---------------
    #---------------
    for i in range(1, x.size):


        #-------------
        if exp_stages>0:
            newlyexposed = interv[i-1] * y[i-1, cS] / population
            J[cE, cI] = newlyexposed
            J[cS, cI] = - newlyexposed
        
        else:
            newlyinfectious = interv[i-1] * y[i-1, cS] / population
            J[cI0, cI] = newlyinfectious
            J[cS, cI] = - newlyinfectious
            J[cT, cI] = detection_rate * newlyinfectious


        
        #-------------
        if x[i]==vacc_start:
            J[cV, cS] = vacc_rate
            J[cS, cS] -= vacc_rate

            J[cS, cV] = vacc_immun
            J[cV, cV] = - vacc_immun
                
        #pJ = pd.DataFrame(J, columns=cCols, index=cCols)
        #print(pJ[cols].loc[cols])
        
        dy = np.dot(J, y[i-1])
        
        #pdy = pd.DataFrame(dy, index=cCols)
        #print(pdy.loc[cols])
        
        y[i] = y[i-1] + dy
        
        #y[i, cI] = np.sum(y[i, cI0:cI0+inf_stages])
        y[i, cI] = 0
        for j in range(inf_stages):
            y[i, cI] += y[i, cI0+j]
        
    return y



In [5]:
import SISV as s

In [6]:

#----------------------------------------------------------
params1 = {
    "population"    : 1e6,
    "exp_stages"    : 1,
    "inf_stages"    : 1,
    "crit_stages"   : 1,
    "test_stages"   : 1,
    "i0"            : 1,
    "death_rate"    : 0.73*1e-2,
    "detection_rate": 10e-2,
    "gamma_exp"     : 1/4,
    "gamma"         : 1/4,
    "gamma_crit"    : 1/35,
    "gamma_pos"     : 1/15,
    "immun"         : 0/360,
    "vacc_start"    : 365,
    "vacc_rate"     : 0.0/365,  #30% of population per year
    "vacc_immun"    : 0/180,
    
    "interv"        : 'piecewise constant',
    "init_beta"     : '',
    'segments'      : 0,
    "beta0"         : 2/4,
    "t1"            : 30,
    "beta1"         : 1.0/4
}

n=300
x = np.arange(n)

y1 = SISV(x, params1)
y2 = SISV_J(x, params1)
y3 = s.SISV(x, params1)

axis_type="log"
p = figure(plot_width=800, plot_height=600 , y_axis_type=axis_type)
#p.line(x[1:], np.diff(y1[:,cI]), line_width=3, line_color='green', line_dash='solid', alpha=0.3)
#p.line(x[1:], np.diff(y2[:,cE]), line_width=1, line_color='blue', line_dash='dashed', alpha=0.3)
#p.line(x[1:], np.diff(y2[:,cI]), line_width=1, line_color='blue', line_dash='solid', alpha=0.3)

p.line(x, y1[:,cI], line_width=5, line_color='green', line_dash='solid', alpha=0.3)

p.line(x, y2[:,cI], line_width=1, line_color='blue', line_dash='solid', alpha=1)

p.line(x, y3[:,cI], line_width=1, line_color='blue', line_dash='solid', alpha=1)

show(p)



In [7]:
n=300
x = np.arange(n)




In [8]:
%%time
y1 = SISV(x, params1)

CPU times: user 8.51 ms, sys: 78 µs, total: 8.59 ms
Wall time: 7.41 ms


In [9]:
%%time
y2 = SISV_J(x, params1)

CPU times: user 5.41 ms, sys: 211 µs, total: 5.63 ms
Wall time: 4.89 ms


In [10]:
population = params1['population']
beta = params1['beta0']

import numpy.linalg as la


#-----------------
# E',I' = J * [E,I]

gamma = params1['gamma']
gamma_exp = params1['gamma_exp']



J1 = [[-gamma_exp, beta],
     [gamma_exp, -gamma]]

e1 = np.sort(la.eig(J1)[0])
e1 = e1[::-1]
print('e1:', e1)

#adjust for a discrete 1-day integration
a1 = e1[0]
aa1 = math.log(1+a1)


yy1 = np.exp(x * aa1)
p.line(x, yy1, line_width=3, line_color='green', line_dash='solid', alpha=0.3)

#-------

J = jacobian(params1, population, beta)
cols = jacobian_cols(params1)
pJ = pd.DataFrame(J, columns=cCols, index=cCols)
print(pJ[cols].loc[cols])
    
e = la.eig(pJ[cols].loc[cols])[0]  #eigenvalues
e = np.sort(e) #ascending order
e = e[::-1] #descending order
print('e:',e)

#adjust for a discrete 1-day integration
a = e[0] #largest eigenvalue
aa = math.log(1+a)

yy = np.exp(x * aa)
p.line(x, yy, line_width=3, line_color='red', line_dash='solid', alpha=0.3)
show(p)


e1: [ 0.10355339 -0.60355339]
      S     E        I0         C    F    R
S   0.0  0.00 -0.500000  0.000000  0.0  0.0
E   0.0 -0.25  0.500000  0.000000  0.0  0.0
I0  0.0  0.25 -0.250000  0.000000  0.0  0.0
C   0.0  0.00  0.001825 -0.028571  0.0  0.0
F   0.0  0.00  0.000000  0.028571  0.0  0.0
R   0.0  0.00  0.248175  0.000000  0.0  0.0
e: [ 0.10355339  0.          0.          0.         -0.02857143 -0.60355339]


In [11]:
def calc_doubling(exp_stages, inf_stages, beta, gamma_exp, gamma ):
      
    population = 1e6
    params1 = {
        "population"    : population,
        "exp_stages"    : exp_stages,
        "inf_stages"    : inf_stages,
        "crit_stages"   : 1,
        "test_stages"   : 1,
        "i0"            : 1,
        "death_rate"    : 0.5*1e-2,
        "detection_rate": 10e-2,
        "gamma_exp"     : gamma_exp,
        "gamma"         : gamma,
        "gamma_crit"    : 1/35,
        "gamma_pos"     : 1/15,
        "immun"         : 0/360,
        "vacc_start"    : 365,
        "vacc_rate"     : 0.0/365,  #30% of population per year
        "vacc_immun"    : 0/180,

        "interv"        : 'piecewise linear',
        "init_beta"     : '',
        'segments'      : 0,
        "beta0"         : beta
    }
    
    J = jacobian(params1, population, beta)
    cols = jacobian_cols(params1) 
    pJ = pd.DataFrame(J, columns=cCols, index=cCols)
    
    #print(pJ[cols].loc[cols])

    e = la.eig(pJ[cols].loc[cols])[0]  #eigenvalues
    e = np.sort(e) #ascending order
    e = e[::-1] #descending order
    
    e = e[np.nonzero(e)]
    #print('nz e:', e)

    #adjust for a discrete 1-day integration
    a = e[0]
    aa = math.log(1+a)
    
    
    return math.log(2)/aa
    

In [12]:
import scipy.optimize as so    


def doubling_to_beta(doubling, exp_stages, inf_stages, gamma_exp, gamma):
    
    def target(beta):
        return doubling - calc_doubling(exp_stages, inf_stages, beta, gamma_exp, gamma )
    
    guess = gamma+math.log(2)/doubling
    
    sol = so.root(target, guess, method='hybr')
    #print(sol)
    return sol.x[0]
    
    

In [13]:
doubling = calc_doubling(1, 1, 0.8*(1/4), 1/4, 1/4)
print(doubling)

beta = doubling_to_beta(doubling, 1, 1, 1/4, 1/4)
print(beta/(1/4))



-25.9142187415677
0.7999999999999997


In [14]:
scenarios = [("blue", 0, 1, 1/4, 1/4), ('red', 1, 1, 1/4, 1/4)]
R0 = np.arange(0.5, 1.2, 0.01)

p = figure(plot_width=800, plot_height=600)

for (color, exp_stages, inf_stages, gamma_exp, gamma) in scenarios: 
    r0s = []
    doubling = []
    checkr0s = []

    for r0 in R0:
        if r0 != 1:
            try:
                d = calc_doubling(exp_stages=exp_stages, inf_stages=inf_stages, beta=gamma*r0, gamma_exp=gamma_exp, gamma=gamma )
                r0s.append(r0)
                doubling.append(d)

                #if r0>1:
                #    r00 = doubling_to_beta(doubling, exp_stages, inf_stages, gamma_exp, gamma)

                #    r00 = r00/gamma
                #    checkr0s.append(r00)
            except:
                pass


    p.line(r0s, doubling, line_width=3, line_color=color, line_dash='solid', alpha=0.3)
    #p.line(r0s, checkr0s, line_width=1, line_color=color, line_dash='solid', alpha=0.3)
    
show(p)

In [53]:
gamma = 1/4

params1 = {
    "population"    : 1e6,
    "exp_stages"    : 1,
    "inf_stages"    : 1,
    "crit_stages"   : 1,
    "test_stages"   : 1,
    "i0"            : 1,
    "death_rate"    : 0.73*1e-2,
    "detection_rate": 10e-2,
    "gamma_exp"     : 1/4,
    "gamma"         : gamma,
    "gamma_crit"    : 1/14,
    "gamma_pos"     : 1/7,
    "immun"         : 0/360,
    "vacc_start"    : 365,
    "vacc_rate"     : 0.0/365,  #30% of population per year
    "vacc_immun"    : 0/180,
    
    "interv"        : 'piecewise constant',
    "init_beta"     : '',
    'segments'      : 0,
    "beta0"         : 3 * gamma,
}

params2 = params1.copy()
params2['segments'] = 1
params2['t1'] = 30
params2['beta1'] = 0.7*gamma


params3 = params2.copy()
params3['exp_stages'] = 3
params3['inf_stages'] = 3
params3['crit_stages'] = 3
#params3['gamma_crit'] = 1/14

n=300
x = np.arange(n)

y1 = SISV_J(x, params1)
y2 = SISV_J(x, params2)
y3 = SISV_J(x, params3)

axis_type="log"
p = figure(plot_width=800, plot_height=600 , y_axis_type=axis_type)

p.line(x[1:], np.diff(y1[:,cF]), line_width=3, line_color='green', line_dash='solid', alpha=0.3)
p.line(x[1:], np.diff(y2[:,cF]), line_width=2, line_color='blue', line_dash='solid', alpha=0.3)
p.line(x[1:], np.diff(y3[:,cF]), line_width=1, line_color='red', line_dash='solid', alpha=1)

#p.line(x, y1[:,cF], line_width=5, line_color='green', line_dash='solid', alpha=0.3)
#p.line(x, y2[:,cF], line_width=1, line_color='blue', line_dash='solid', alpha=1)

show(p)
